In [1]:
import csv
import os
import re
#import time
import pandas as pd

In [2]:
f = open("Host_DB/Proteins_HostDB_Hsapiens/HostDB-68_HsapiensREF_AnnotatedProteins.fasta", "r")

print(f.readlines(10))

f.close()

['>ENSP00000003583 | transcript=ENST00000003583 | gene=ENSG00000001460 | organism=Homo_sapiens_REF | gene_product=sperm tail PG-rich repeat containing 1 [Source:HGNC Symbol;Acc:HGNC:28070] | transcript_product=sperm tail PG-rich repeat containing 1 [Source:HGNC Symbol;Acc:HGNC:28070] | location=hsapREF_chr1:24358543-24401366(-) | protein_length=287 | sequence_SO=chromosome | SO=protein_coding_gene | is_pseudo=false\n']


In [13]:
input_folder = output_folder = "Host_DB/Proteins_HostDB_Hsapiens"

# Obtener la lista de archivos .fasta en la carpeta
fasta_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".fasta")])

# Expresión regular mejorada para capturar solo los transcript que empiezan con ENST
pattern = re.compile(r'transcript=(ENST[0-9]+)')

# Procesar cada archivo .fasta
for fasta_file in fasta_files:
    input_path = os.path.join(input_folder, fasta_file)

    # Obtener el número del archivo (usando una expresión regular para extraer el número de HostDB)
    match = re.search(r'HostDB-(\d+)', fasta_file)
    if match:
        file_number = match.group(1)  # Número del archivo, como '49'
    else:
        file_number = 'unknown'  # Si no se encuentra el número, usar un valor por defecto

    output_path = os.path.join(output_folder, f"HostDB-{file_number}_Hsapiens.txt")

    transcript_ids = set()  # Usar un conjunto para asegurar unicidad

    with open(input_path, "r") as f:
        for line in f:
            match = pattern.search(line)
            if match:
                transcript_ids.add(match.group(1).strip())  # Limpiar espacios en blanco extra

    # Guardar los resultados en un archivo .txt con el formato adecuado
    with open(output_path, "w") as out_f:
        out_f.write("Transcript_ID\tTaxon_ID\n")
        for transcript_id in sorted(transcript_ids):  # Ordenar por claridad
            out_f.write(f"{transcript_id}\t9606\n")

In [6]:
input_folder = "Host_DB/Proteins_HostDB_Hsapiens"
combined_output_path = "Host_DB/Proteins_HostDB_Hsapiens/combined_transcripts_hostDB_Hsapiens.txt"

# Obtener todos los archivos .txt en la carpeta
txt_files = [f for f in os.listdir(input_folder) if f.endswith(".txt")]

# Crear una lista para almacenar todas las líneas
combined_data = []

# Procesar cada archivo .txt
for txt_file in txt_files:
    input_path = os.path.join(input_folder, txt_file)
    
    with open(input_path, "r") as f:
        # Agregar todas las líneas, omitiendo el encabezado de los archivos adicionales
        if combined_data:
            combined_data.extend(f.readlines()[1:])  # Omitir el encabezado
        else:
            combined_data.extend(f.readlines())  # Incluir el encabezado del primer archivo

# Guardar el archivo combinado
with open(combined_output_path, "w") as combined_file:
    combined_file.writelines(combined_data)

In [7]:
# Ruta del archivo original y del archivo de salida
input_file = "Host_DB/Proteins_HostDB_Hsapiens/combined_transcripts_HostDB_Hsapiens.txt"
output_file = "Host_DB/Proteins_HostDB_Hsapiens/updated_transcripts_HostDB_Hsapiens.txt"

# Leer el archivo original
with open(input_file, "r") as f:
    lines = f.readlines()

# Abrir el archivo de salida para escribir los datos actualizados
with open(output_file, "w") as out_f:
    # Escribir el encabezado (sin modificarlo)
    out_f.write(lines[0])

    # Iterar sobre las líneas, comenzando desde la segunda (omitiendo el encabezado)
    for line in lines[1:]:
        columns = line.strip().split("\t")  # Separar por tabulaciones
        columns[0] = "9606." + columns[0]  # Agregar el prefijo a Transcript_ID
        out_f.write("\t".join(columns) + "\n")  # Escribir la línea modificada

In [8]:
# Ruta del archivo original y del archivo de salida
input_file = "Host_DB/Proteins_HostDB_Hsapiens/updated_transcripts_HostDB_Hsapiens.txt"
output_file = "Host_DB/Proteins_HostDB_Hsapiens/modified_transcripts_HostDB_Hsapiens.txt"

# Leer el archivo original
with open(input_file, "r") as f:
    lines = f.readlines()

# Abrir el archivo de salida para escribir los datos actualizados
with open(output_file, "w") as out_f:
    # Escribir el encabezado (sin modificarlo)
    out_f.write(lines[0])

    # Iterar sobre las líneas, comenzando desde la segunda (omitiendo el encabezado)
    for line in lines[1:]:
        columns = line.strip().split("\t")  # Separar por tabulaciones
        columns[0] = columns[0].replace("9606.ENST", "9606.ENSP")  # Reemplazar ENST por ENSP
        out_f.write("\t".join(columns) + "\n")  # Escribir la línea modificada

In [14]:
# Cargar archivo con manejo de errores
def cargar_csv(ruta, sep='\t'):
    try:
        df = pd.read_csv(ruta, sep=sep)
        return df
    except FileNotFoundError:
        print(f"Error: El archivo {ruta} no se encontró.")
        return None
    except pd.errors.ParserError:
        print("Error: Hubo un problema al analizar el archivo.")
        return None

# Comparar las columnas y guardar las coincidencias
def comparar_y_guardar(df_transcripts, df_string, salida):
    if df_transcripts is not None and df_string is not None:
        # Filtrar filas donde las columnas de Transcript_ID y string_protein_id coinciden
        coincidencias = df_transcripts[df_transcripts['Transcript_ID'].isin(df_string['string_protein_id'])]
        
        # Obtener los valores de preferred_name correspondientes a las coincidencias
        resultados = df_string[df_string['string_protein_id'].isin(coincidencias['Transcript_ID'])]['preferred_name']
        
        # Guardar los resultados sin encabezado
        resultados.to_csv(salida, index=False, header=False, sep='\t')
    else:
        print("No se pudo realizar la comparación debido a un error al cargar los archivos.")

# Rutas de los archivos de entrada
ruta_transcripts = "Host_DB/Proteins_HostDB_Hsapiens/modified_transcripts_HostDB_Hsapiens.txt"
ruta_string = "string_data/9606.protein.info.v12.0.txt"

# Ruta del archivo de salida
salida_tsv = "Host_DB/Proteins_HostDB_Hsapiens/Matched_Transcripts_StringDB.tsv"

# Cargar los archivos
df_transcripts = cargar_csv(ruta_transcripts)
df_string = cargar_csv(ruta_string)

# Comparar y guardar resultados
comparar_y_guardar(df_transcripts, df_string, salida_tsv)